In [3]:
import os
import os.path as osp
import sys
import logging
import csv
import io
import base64
import ast

from tqdm import tqdm
import json


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas  
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator



from flask import Flask, jsonify, request, render_template
from flask_cors import CORS 
from ragutils import ChatGpt, ragChat
from langchain_openai import ChatOpenAI
import importlib
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document




In [15]:
modelDict = dict()
with open ('./shapetalk.csv', 'r', encoding='gbk') as f:
    counter = 0
    print('initializing')
    reader = csv.reader(f)
    for row in tqdm(reader):
        if (row[6] not in modelDict):
            modelDict[row[6]] = set()
        for i in range(1, 5):
            if (row[i] != ''):
                modelDict[row[6]].add(row[i])
        counter += 1

modelDict.pop('ShapeNet')
modelDict.pop('source_object_class')
print (modelDict.keys())

# for sentence in list(modelDict['vase']):
#     vaseDocs.append(Document(page_content=sentence))
#     vaseId.append('vase' + str(i))
#     i += 1
# 构造类名->描述文本，处理描述解析任务
source_knowledge_dict = {}
for key in modelDict.keys():
    print (key + '-' + str(len(list(modelDict[key]))))
    source_knowledge_dict[key] = []
    sentenCounter = 0
    longSentences = ''
    for sentence in modelDict[key]:
        # 划分一百句为一个整体
        if (sentenCounter == 100):
            source_knowledge_dict[key].append(longSentences)
            longSentences = ''
            sentenCounter = 0
        longSentences += sentence + '\n'
        sentenCounter += 1
    if (sentenCounter > 0):
        source_knowledge_dict[key].append(longSentences)
        longSentences = ''
        sentenCounter = 0
        
# 构造类名->描述句子，将句子转换为embedding，每类只取200句
discriptionDocs = {}
for key in modelDict.keys():
    sentenCounter = 0
    discriptionDocs[key] = []
    for sentence in modelDict[key]:
        discriptionDocs[key].append(Document(page_content=sentence))
        sentenCounter += 1
        if (sentenCounter > 200):
            sentenCounter = 0
            break

# print (discriptionDocs)
# print (source_knowledge_dict)
# source_class = 'cap'
# for sentence in modelDict['cap']:
#     source_knowledge_cap += sentence + '\n'
# print(source_knowledge_cap)

initializing


130343it [00:00, 145688.24it/s]


dict_keys(['faucet', 'cap', 'plant', 'cabinet', 'bookshelf', 'knife', 'table', 'skateboard', 'mug', 'vase', 'pistol', 'flowerpot', 'clock', 'chair', 'bathtub', 'bottle', 'display', 'bag', 'trashbin', 'scissors', 'person', 'helmet', 'bowl', 'airplane', 'guitar', 'dresser', 'bed', 'sofa', 'bench', 'lamp'])
faucet-7041
cap-2094
plant-2687
cabinet-2578
bookshelf-8247
knife-4915
table-46480
skateboard-1980
mug-1491
vase-7016
pistol-3660
flowerpot-5625
clock-6116
chair-39479
bathtub-8594
bottle-4149
display-8047
bag-2726
trashbin-4056
scissors-715
person-1768
helmet-1451
bowl-1852
airplane-24645
guitar-6823
dresser-12808
bed-6860
sofa-29684
bench-14045
lamp-36758


In [16]:
# 这里会消耗巨量tokens，慎用
embed_model = OpenAIEmbeddings(base_url="https://api3.apifans.com/v1")
vectorstore = {}
for key in discriptionDocs.keys():
    print(f"\r当前进度：{key}-{len(discriptionDocs[key])}", end='')
    vectorstore[key] = Chroma.from_documents(documents=discriptionDocs[key], embedding=embed_model, collection_name="model_discription_embed")
print (vectorstore)

当前进度：lamp-201101111{'faucet': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732d8436d0>, 'cap': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732ba851b0>, 'plant': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732d88dff0>, 'cabinet': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732ba87490>, 'bookshelf': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732d88e0b0>, 'knife': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732ba87790>, 'table': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732d88eda0>, 'skateboard': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732ba86980>, 'mug': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732d88db10>, 'vase': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732ba87370>, 'pistol': <langchain_community.vectorstores.chroma.Chroma object at 0x7f732d88e7d0>, 'flowerpot': <langchain_community.vectorstores.c

In [17]:
print(discriptionDocs['vase'])

[Document(metadata={}, page_content='The vase gets narrower as it goes from the base to the top'), Document(metadata={}, page_content='it has sharp edges and corners'), Document(metadata={}, page_content="It doesn't look like a bell."), Document(metadata={}, page_content='The target is filled with flowers inside it.'), Document(metadata={}, page_content='The body ofthe target is thinner and smaller.'), Document(metadata={}, page_content='The lips are more pronounced.'), Document(metadata={}, page_content='The mouth ofthe target hasa smaller opening.'), Document(metadata={}, page_content="It looks like an upside down chess Hershey's Kiss."), Document(metadata={}, page_content='The target has a shorter base.'), Document(metadata={}, page_content='Has a wider lip'), Document(metadata={}, page_content='There is a neck and it is longer'), Document(metadata={}, page_content='The body is fatter.'), Document(metadata={}, page_content='You can see through more of it.'), Document(metadata={}, pa

In [19]:
# query = "The lips"
# result = vectorstore['vase'].similarity_search(query ,k = 5)
# result
print (vectorstore)

# with open('discription_embedding.json', 'w') as f:
#     json.dump(vectorstore, f, indent=4)

TypeError: Object of type Chroma is not JSON serializable

In [3]:


for key in source_knowledge_dict.keys():
    print (len(source_knowledge_dict[key]))

71
21
27
26
83
50
465
20
15
71
37
57
62
395
86
42
81
28
41
8
18
15
19
247
69
129
69
297
141
368


In [6]:
ragClient = ragChat()
source_knowledge = 'The target\'s foot is more round and the vase is with builtiful color, the top of it has a longger head\nThe top of the vase is open\nThe vase has flowers in it'
modelClass = 'vase'
prompt_template1 = f"""以下内容为对一个{modelClass}的描述集合，将以下内容中的每一句按照格式以键值字典的方式拆分，键为该物体的部位，如果是对整体的描述则直接用{modelClass}作为键，值为该部位对应的描述。请返回给我一个键值字典列表，列表中每一项为一个键值字典。不要使用代码格式返回:

内容:
{source_knowledge}

"""
ragClient.pushUserMessage(prompt_template1)

# 以下操作消耗大量tokens
answer = ragClient.getandpushGptMsg()
ragClient.MessageHistory()

User: 以下内容为对一个vase的描述集合，将以下内容中的每一句按照格式以键值字典的方式拆分，键为该物体的部位，如果是对整体的描述则直接用vase作为键，值为该部位对应的描述。请返回给我一个键值字典列表，列表中每一项为一个键值字典。不要使用代码格式返回:

内容:
The target's foot is more round and the vase is with builtiful color, the top of it has a longger head
The top of the vase is open
The vase has flowers in it


*************************************************
AI: [
  {"foot": "more round"},
  {"vase": "with beautiful color"},
  {"top": "has a longer head"},
  {"top": "is open"},
  {"vase": "has flowers in it"}
]
*************************************************


In [7]:
# ragClient = ragChat()

# dict_keys(['faucet', 'cap', 'plant', 'cabinet', 'bookshelf', 'knife', 'table', 
# 'skateboard', 'mug', 'vase', 'pistol', 'flowerpot', 'clock', 'chair', 'bathtub', 
# 'bottle', 'display', 'bag', 'trashbin', 'scissors', 'person', 'helmet', 'bowl', 
# 'airplane', 'guitar', 'dresser', 'bed', 'sofa', 'bench', 'lamp'])
part2Discription = {}
for modelClass in source_knowledge_dict.keys():
    # 先不处理长度大于一百万的句子，tokens撑不住
# modelClass = 'faucet'
    # if (len(source_knowledge_dict[modelClass]) < 10):
    discription = dict()
    index = 0
    for sentences in source_knowledge_dict[modelClass]:
        # 为了节省tokens，每个类只取前200句得了
        if (index > 1):
            break
        print(f"\r当前进度：{modelClass}-{index}", end='')
        index += 1
        prompt_template1 = f"""以下内容为对一个{modelClass}的描述集合，将以下内容中的每一句按照格式以键值字典的方式拆分，键为该物体的部位，如果是对整体的描述则直接用{modelClass}作为键，值为该部位对应的描述。请返回给我一个python键值字典列表，列表中每一项为一个键值字典。不要使用代码格式返回:
    
        内容:
        {sentences}
        """
        ragClient.pushUserMessage(prompt_template1)
        answer = ragClient.getGptMsg()
        parsedStr = json.loads(answer)
        for dictEle in parsedStr:
            for key in dictEle.keys():
                if key in discription:
                    discription[key].add(dictEle[key])
                else:
                    discription[key] = set()
                    discription[key].add(dictEle[key])
        ragClient.deleteMessage()
    for key in discription.keys():
        discription[key] = list(discription[key])
    part2Discription[modelClass] = discription



当前进度：lamp-11-1111

In [10]:
print (part2Discription['lamp'])

{'top': ['pointy', 'hat shaped', 'has triangles and a ball shape in the middle', 'is larger than its bottom'], 'feet': ['3 small under the square base'], 'tube': ['is curved', 'has a rounded bend at the top', 'does not extend out beyond the footprint of the base', 'longest is thinner', 'thinner'], 'shade': ['does not have a circular shape', 'contains a tassel', 'has a taper square shape', 'is tulip shaped', 'extended out by thin chords', 'very small and scooped shaped', 'Pembroke Empire shape', 'longer and wider', 'comes to more of a point', "isn't solid", 'has rod in the center', 'has a conical top', 'pointed at the top', 'conical shade points the light straight up', 'does not have a scalloped top', 'is longer', 'is a separate part, not part of the lamp itself', 'has gaps in it', 'rounder, globular', 'does not have a shape like a half cylinder', 'tapered drum', 'not as wide', 'smaller opening', 'has an open top', 'resembles a book', 'has sphere in the middle', 'giant', 'long, vertical

In [53]:
# answer = '''[
#   {"foot": "more round"},
#   {"vase": "with beautiful color"},
#   {"top": "has a longer head"},
#   {"top": "is open"},
#   {"vase": "has flowers in it"},
#   {"top": "has a longer head"}]'''
# answer += ''
# print (answer)
# print (answer)
# parsedStr = json.loads(answer)

# vasePart2Discription = {}
# for dictEle in parsedStr:
#     for key in dictEle.keys():
#         if key in vasePart2Discription:
#             vasePart2Discription[key].add(dictEle[key])
#         else:
#             vasePart2Discription[key] = set()
#             vasePart2Discription[key].add(dictEle[key])
# # print (vasePart2Discription)
# for key in vasePart2Discription.keys():
#     vasePart2Discription[key] = list(vasePart2Discription[key])
with open('Part2Discription', 'w') as f:
    json.dump(part2Discription, f, indent=4)

In [8]:
# 1：这个是基于用户问答做反馈
ragClient = ragChat()
Part2Discription = {}
with open('Part2Discription', 'r') as f:
    Part2Discription = json.load(f)
query = '请问我可以对Brim做什么改动？'
prompt_template2 = f"""请基于以下内容回答问题，并返回对应的条目原文，不要翻译

内容:
{Part2Discription['cap']}
query:
{query}
"""
ragClient.pushUserMessage(prompt_template2)
answer = ragClient.getandpushGptMsg()
print (answer)

# 2：对用户输入prompt进行编码，做embedding相似的编辑方向推荐

'brim': ['thinner vertically', 'is taller vertically', 'circular all the way around', 'goes around the entire hat', 'flat', 'wider and asymmetrical', 'thin', 'is thinner', 'turns downward', 'smaller diameter', 'is puffed out', 'extends out and is more squarish at the front', 'thicker', 'only covers the front', 'thinner', 'diameter is smaller', 'rounded', 'faces downward', 'does not go all the way around', 'thinner around hat']
